<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/course4/week4/Art_Generation_with_Neural_Style_Transfer_v3a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning & Art: Neural Style Transfer

이번 과제에서, 여러분은[ Gatys et al. (2015).](https://arxiv.org/abs/1508.06576) 이 개발한 알고리즘인 Neural Style Transfer에 대해서 배워볼 것입니다.

** 이번 과제에서 여러분은 : **
- Neural style transfer 알고리즘을 직접 구현합니다.
- 위 알고리즘을 사용해서 참신한 예술 작품을 생성합니다.

이전까지 공부했던 대부분의 인공신경망 알고리즘은 파라미터 세트를 얻기 위해 비용 함수를 최적화합니다. 그에 반해, 이번에 새로 배울 Neural style transfer에서는 픽셀 값을 얻기 위해 비용 함수를 최적화합니다.

In [ ]:
import os
import sys
import scipy.io
import scipy.misc
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from PIL import Image
from nst_utils import *
import numpy as np
import tensorflow as tf
import pprint
%matplotlib inline

## 1 - Problem Statement

Neural Style Transfer(NST)는 딥러닝에서 가장 흥미로운 몇 가지 기법 중 하나입니다. 아래 그림에서 볼 수 있듯이, NST 알고리즘은 두 이미지를 하나로 합칩니다. 좀더 정확히는 **"contents" 이미지 (C) 와 "style" 이미지 (S) 를 합쳐 "geneated" 이미지 (G) 를 만드는 알고리즘입니다.**

즉 생성된 이미지 G는 "content" 이미지 C와 "style" 이미지 S가 결합된 형태입니다.

이 예에서는 대표적인 인상파 화가인 클로드 모네의 그림 (style 이미지 S)과 파리 루브르 박물관의 이미지 (content 이미지 C)를 결합한 이미지를 생성합니다.

<img src="arts/louvre_generated.png" style="width:750px;height:200px;">

어떻게 이런 작품을 만들 수 있는지 알아봅시다.

## 2 - Transfer Learning

NST (Neural Style Transfer)는 이전에 훈련 된 컨볼 루션 신경망을 사용하며 그 위에서 이루어집니다. 사전에 훈련 된 네트워크를 새로운 작업에 적용하는 아이디어를 `transfer learning` 이라고 합니다.

이번 과제에서는 [원본 NST 논문](https://arxiv.org/abs/1508.06576)에 맞게 VGG 네트워크를 사용합니다. 특히 이번에는 19개의 은닉층을 가진 VGG-19 버전을 사용할 것입니다. 이 모델은 이미 매우 큰 ImageNet 데이터베이스에서 학습되었으므로, 다양한 하위 레벨 feature(얕은 레이어)와 상위 레벨 feature(깊은 레이어)를 인식하는 방법을 배웠습니다.

아래 코드를 실행하여 VGG 모델에서 학습된 파라미터를 다운로드합니다. 몇 초 정도 걸릴 수 있습니다.

In [ ]:
pp = pprint.PrettyPrinter(indent=4)
model = load_vgg_model("pretrained-model/imagenet-vgg-verydeep-19.mat")
pp.pprint(model)

- 위 모델은 python 딕셔너리에 저장되어 있습니다.
- 딕셔너리 자료구조는 각 레이어에 대해 key-value 쌍의 데이터를 가지고 있습니다.
- 'key' 값이 변수명이고, 'value'는 해당 레이어의 텐서 값을 의미합니다.

#### Assign input image to the model's input layer
이 네트워크를 사용해서 이미지를 분석하려면 이미지를 모델에 투입하기만 하면 됩니다. TensorFlow 에서는 [tf.assign](https://www.tensorflow.org/api_docs/python/tf/assign) 함수를 사용하여 이를 수행 할 수 있습니다. 특히 다음과 같이 할당 기능을 사용합니다.
```python
model["input"].assign(image)
```
이 코드는 이미지를 모델에 대한 입력으로 할당합니다.

#### Activate a layer
그런 다음 특정 레이어(예 : 이 이미지에서 네트워크가 실행될 때 레이어 `4_2`) 의 활성화 변수에 액세스하려면 다음과 같이 텐서 `conv4_2`에서 TensorFlow 세션을 실행합니다.
```python
sess.run(model["conv4_2"])
```


## 3 - Neural Style Transfer(NST)

지금부터 다음 3단계를 거쳐서 Neural Style Transfer(NST) 알고리즘을 구현해보도록 하겠습니다.

- content 비용함수 $J_{content}(C,G)$ 구현하기
- style 비용함수 $J_{style}(S,G)$ 구현하기
- 두 비용함수를 한데 모아 $J(G) = \alpha J_{content}(C,G) + \beta J_{style}(S,G)$ 만들기

### 3.1 - Computing the content cost

이번 과제에서, 컨텐츠(content) 이미지 C는 파리 루브르 박물관 사진입니다. 아래 코드를 실행시켜서 루브르 박물관 이미지가 어떻게 생겼는지 확인해보세요.

In [ ]:
content_image = scipy.misc.imread("images/louvre.jpg")
imshow(content_image);

컨텐츠 이미지 (C)는 구름이 적당히 드리운 하늘 아래 파리의 건축물들 사이에 둘러쌓인 루브르 박물관의 유리 피라미드를 담고 있습니다. 

#### 3.1.1 - Make generated image G match the content of image C

**Shallower versus deeper layers**

- 컨볼루션 신경망의 얕은 레이어 부분은 모서리나 단순한 텍스쳐 등 저수준의 feature를 감지합니다
- 깊은 레이어는 더 복잡한 텍스처 및 물체의 클래스와 같은 더 높은 수준의 feature를 감지하는 경향이 있습니다.

<br>

**Choose a "middle" activation layer $a^{[l]}$**

우리는 "generated" 이미지 G가 입력 content 이미지 C와 유사한 내용을 갖기를 원합니다. 이미지의 content를 나타내기 위해 특정 레이어의 활성화를 선택했다고 가정합니다.

- 실제로 너무 얕거나 너무 깊지 않은 네트워크 중간에 있는 레이어를 선택하면 시각적으로 가장 만족스러운 결과를 얻을 수 있습니다.
- (이 과제를 다 마친 후 다시 돌아와서 다른 레이어를 사용하여 실험하여 결과가 어떻게 다른지 확인해보세요.)

<br>

**Forward propagate image "C"**

- 이미지 C를 사전 훈련된 VGG 네트워크의 입력으로 집어넣고, forward propagation을 수행합니다.
- $a^{(c)}$를 위에서 선택한 특정 은닉층의 활성화 변수라고 해봅시다.(동영상 강의에선 해당 변수를 $a^{[l](C)}$로 적었습니다, 하지만 과제에서는 편의를 위해 위첨자 $[l]$을 생략하겠습니다.) 이 변수는 $n_H \times n_W \times n_C$의 텐서입니다.

<br>

**Forward propagate image "G"**

- 위의 과정을 이미지 G에 대해서도 반복적으로 수행하세요. G를 입력으로 하여 forward propagation을 수행합니다.
- $a^{(G)}$를 위에 상응하는 은닉층의 활성화 변수로 설정합니다.

<br>

**Content Cost Function $J_{content}(C, G)$**

content 비용 함수를 다음과 같이 정의할 수 있습니다.

$$J_{content}(C,G) =  \frac{1}{4 \times n_H \times n_W \times n_C}\sum _{ \text{all entries}} (a^{(C)} - a^{(G)})^2\tag{1} $$

- 여기서, $n_H, n_W, n_C$는 위에서 여러분이 선택한 은닉층의 각 높이, 너비, 채널의 수이며, 정규화된 방식으로 표현됩니다.
- 보다 명확히 하기 위해, $a^{(C)}$ 및 $a^{(G)}$ 는 히든 레이어의 활성화에 해당하는 3D 볼륨입니다.
- 비용 $J_{content}(C, G)$ 를 계산하려면 아래 그림과 같이 3D 볼륨을 2D 매트릭스로 펼치는 것이 편리 할 수도 있습니다.
- 기술적으로는 $J_{content}$ 를 계산하는 데 이 '펼치는' 단계가 필요하지 않지만 나중에 style 비용 $J_ {style}$를 계산하기 위해 유사한 작업을 수행해야 할 때 좋은 방법이 될 것입니다.

<img src="arts/NST_LOSS.png" style="width:800px;height:400px;">

**연습 문제** : Tensorflow를 사용해서 "content cost" 를 계산하세요.

**지시 사항** : 위 함수를 구현하기 위해서는 아래 3단계를 따르세요.

1. `a_G`로부터 배열의 차원을 구합니다 : 
  - tensor `X` 로부터 차원을 구해내려면, `X.get_shape().as_list()` 코드를 사용합니다.
2. `a_C`와 `a_G`를 위 그림에 나온대로 펼쳐보세요.
  - `tf.transpose` 함수와 `tf.reshape` 함수를 사용하세요.
3. content cost를 계산하세요.
  - `tf.reduce_sum`, `tf.square`, `tf.subtract` 함수를 사용하세요.

**"Unrolling(펼치기)" 연산을 위한 추가적인 힌트**


- 텐서를 펼치기 위해 모양을 $(m, n_H, n_W, n_C) $에서 $ (m, n_H \times n_W, n_C)$로 변경하려고합니다.
- `tf.reshape (tensor, shape)` 는 원하는 출력 shape를 나타내는 정수 목록을 인자로 받습니다.
- `shape` 매개 변수의 경우 `-1` 은 출력 텐서가 원래 텐서의 모든 값을 포함하도록 올바른 차원 크기를 선택하도록 합니다.
- 따라서 `tf.reshape(a_C, shape = [m, n_H * n_W, n_C])` 는 `tf.reshape (a_C, shape = [m, -1, n_C])` 와 동일한 결과를 리턴합니다.
- 차원을 다시 정렬하려면 `tf.transpose (tensor, perm)` 를 사용할 수 있습니다. 여기서 `perm` 은 차원의 원래 색인을 포함하는 정수 목록입니다.
- 예를 들어 `tf.transpose (a_C, perm = [0,3,1,2])`는 차원을 $(m, n_H, n_W, n_C)$ 에서 $(m, n_C, n_H, n_W)$ 로 변경합니다.
- 텐서를 펼치는 방법은 이외에도 여러 가지가 있습니다.
- 이 경우 텐서를 'unroll'하기 위해 `tf.transpose` 를 사용할 필요는 없지만 마주하게 될 다른 상황에 대비해 연습하고 이해하는 데 유용한 함수입니다.

In [ ]:
# GRADED FUNCTION: compute_content_cost

def compute_content_cost(a_C, a_G):
    """
    Computes the content cost
    
    Arguments:
    a_C -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing content of the image C 
    a_G -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing content of the image G
    
    Returns: 
    J_content -- scalar that you compute using equation 1 above.
    """
    
    ### START CODE HERE ###
    # Retrieve dimensions from a_G (≈1 line)
    m, n_H, n_W, n_C = None
    
    # Reshape a_C and a_G (≈2 lines)
    a_C_unrolled = None
    a_G_unrolled = None
    
    # compute the cost with tensorflow (≈1 line)
    J_content = None
    ### END CODE HERE ###
    
    return J_content

In [ ]:
tf.reset_default_graph()

with tf.Session() as test:
    tf.set_random_seed(1)
    a_C = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    a_G = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    J_content = compute_content_cost(a_C, a_G)
    print("J_content = " + str(J_content.eval()))

**모범 답안**:
<table>
    <tr>
        <td>
            <b>J_content</b>
        </td>
        <td>
           6.76559
        </td>
    </tr>
</table>

#### 기억해야 할 사항
- content cost는 신경망의 은닉층 활성화를 취하고 $a ^ {(C)}$와 $ a^{(G)}$가 얼마나 다른지 측정합니다.
- 나중에 content cost를 최소화하면 $G$에 $C$와 유사한 컨텐츠가 있는지 확인하는 데 도움이됩니다.

### 3.2 - Computing the style cost

이번 예제에서는 다음 스타일 이미지를 사용합니다.

In [ ]:
style_image = scipy.misc.imread("images/monet_800600.jpg")
imshow(style_image);

위 그림은 [인상파](https://en.wikipedia.org/wiki/Impressionism) 스타일로 그려졌습니다.

이제 "style" 비용 함수 $ J_{style}(S, G)$ 를 정의하는 방법을 살펴 보겠습니다.

#### 3.2.1 - Style matrix

**Gram matix**

- style matrix는 "Gram matrix" 라는 이름으로도 불립니다.
- 선형 대수학에서, $(v_{1},\dots ,v_{n})$인 Gram matrix G는 엔트리가 ${\displaystyle G_{ij} = v_{i}^T v_{j} = np.dot(v_{i}, v_{j})}$인 내적 행렬입니다. 
- 즉 $G_{ij}$는 $v_i$가 $v_j$와 얼마나 유사한지 비교합니다. 만약 두 값이 매우 유사하다면, 더 큰 내적을 가진다고 예상할 수 있고, 따라서 $G_{ij}$가 더 커질 것입니다.

<br>

**Two meanings of the variable G**

- 아래를 읽어보면 현재 사용하고 있는 변수 이름에 불필요한 충돌이 있음을 알 수 있습니다. 이번 과제는 참고 문헌에서 사용되는 일반적인 용어를 따르고 있습니다.
- $G$는 Style matrix(Gram matrix)를 의미합니다.
- $G$는 생성된 이미지를 나타내기도 합니다.
- 이번 과제에서는 $G_{gram}$을 사용하여 Gram matrix를 나타내고, $G$를 사용하여 생성된 이미지를 나타내도록 하겠습니다.

<br>

**Compute G_{gram}**

Neural Style Transfer(NST)에서, 펼쳐진 버전의 행렬을 자기 자신의 전치 행렬과 곱하는 연산을 수행함으로서 Style matrix를 구할 수 있습니다

<img src="arts/NST_GM.png" style="width:900px;height:300px;">

$$\mathbf{G}_{gram} = \mathbf{A}_{unrolled} \mathbf{A}_{unrolled}^T$$

$G_{(gram),i,j}$ : **correlation(상관 계수)**

위 연산의 결과값은 $(n_C, n_C)$ 의 shape를 가진 행렬로, $n_C$는 채널(필터)의 개수를 의미합니다. $G_{(gram), i, j}$는 필터 i의 활성화 값이 필터 j의 활성화 값과 얼마나 비슷한지를 측정합니다.

<br>

$G_{(gram), i, i}$ : **prevalence of patterns or textures**

- 행렬의 대각선에 해당하는 element인 $G _{(gram)ii}$ 는 필터 $i$의 활성화 정도를 측정합니다.
- 예를 들어 $i$ 필터가 특정 이미지에서 수직인 텍스처를 감지한다고 가정합니다. 그런 다음 $G_{(gram) ii}$는 이미지 전체에 수직 텍스처가 얼마나 일반적으로 나타나는지 측정합니다.
- $G_{(gram)ii}$가 크면 이미지에 수직인 텍스처가 많음을 의미합니다.

스타일 매트릭스 $G_{gram}$는 서로 다른 유형의 featuree들의 보급률(널리 퍼진 정도)과, 서로 다른 feature가 얼마나 같이 발생하는지의 양을 포착하여 이미지의 스타일을 측정합니다.

**연습 문제**:
- 텐서플로우를 사용해, 특정 행렬 A의 Gram matrix를 계산하는 함수를 구현해보세요.
- Gram matrix를 구하는 공식은 다음과 같습니다 : $G_A = AA^{T}$
- `matmul` 함수와 `transpose` 함수를 사용해야 합니다.

In [ ]:
# GRADED FUNCTION: gram_matrix

def gram_matrix(A):
    """
    Argument:
    A -- matrix of shape (n_C, n_H*n_W)
    
    Returns:
    GA -- Gram matrix of A, of shape (n_C, n_C)
    """
    
    ### START CODE HERE ### (≈1 line)
    GA = None
    ### END CODE HERE ###
    
    return GA

In [ ]:
tf.reset_default_graph()

with tf.Session() as test:
    tf.set_random_seed(1)
    A = tf.random_normal([3, 2*1], mean=1, stddev=4)
    GA = gram_matrix(A)
    
    print("GA = \n" + str(GA.eval()))

**모범 답안**:

<table>
    <tr>
        <td>
            <b>GA</b>
        </td>
        <td>
           [[  6.42230511  -4.42912197  -2.09668207] <br>
 [ -4.42912197  19.46583748  19.56387138] <br>
 [ -2.09668207  19.56387138  20.6864624 ]]
        </td>
    </tr>

</table>

#### 3.2.2 - Style cost

여러분의 목표는, "Style" 이미지 S에 대한 Gram matrix와 "generated" 이미지 G의 Gram matrix 사이의 거리를 최소화하는 것입니다.

- 지금까지 우리는 단 하나의 은닉층 $a^{[l]}$ 만을 사용했습니다.
- 해당 레이어에 상응하는 style 비용함수는 다음과 같이 정의됩니다.

$$J_{style}^{[l]}(S,G) = \frac{1}{4 \times {n_C}^2 \times (n_H \times n_W)^2} \sum _{i=1}^{n_C}\sum_{j=1}^{n_C}(G^{(S)}_{(gram)i,j} - G^{(G)}_{(gram)i,j})^2\tag{2} $$

- $G_{gram}^{(S)}$는 style 이미지에 대한 Gram matrix를 의미합니다.
- $G_{gram}^{(G)}$는 generated 이미지에 대한 Gram matrix를 의미합니다.
- 이 비용은 신경망의 특정 히든 레이어인 $a^{[l]}$에 대한 활성화 값을 사용하여 계산된다는 것을 기억하세요.

**연습 문제** : 단일 신경망에 대한 style 비용 함수를 작성하세요.

**지시 사항** : 함수 구현에 있어 다음 세 단계를 따르세요.

1. 특정 은닉층의 활성화 값인 a_G에 대한 차원을 구하세요.
  - 텐서 X로부터 차원을 구하기 위해선 `X.get_shape().as_list()` 코드를 사용합니다.
2. 스타일 이미지와 생성된 이미지의 해당 은닉층에서의 활성화변수인 `a_S`와 `a_G`를 2D 행렬로 펼쳐보세요. 아래에 그림에 설명된대로 하면 됩니다("computing the content const"와 "style matrix" 섹션을 참조하세요)
  - [tf.transpose](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/transpose) 함수와 [tf.reshape](https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/reshape) 함수를 사용하세요.
3. 이미지 S와 G의 style 행렬을 계산합니다. (이전에 작성한 함수를 사용하세요.)
4. style cost를 계산합니다.
     - [tf.reduce_sum](https://www.tensorflow.org/api_docs/python/tf/reduce_sum), [tf.square](https://www.tensorflow.org/api_docs/python/tf/square) 및 [tf.subtract](https://www.tensorflow.org/api_docs/python/tf/subtract) 함수를 사용하세요.

**Additional Hints**
- 활성화 변수 행렬의 차원이 $(m, n_H, n_W, n_C)$ 인 반면 원하는 펼쳐진 행렬의 shape는 $(n_C, n_H * n_W)$이므로 필터 차원 $n_C$의 순서가 변경됩니다. 따라서`tf.transpose`를 사용하여 필터 차원의 순서를 변경할 수 있습니다.
- $\mathbf{G}_{gram} = \mathbf{A}_{} \mathbf{A}_{}^T$ 행렬곱을 수행하기 위해서, `tf.transpose`함수에 대해 `perm` 파라미터를 지정해야합니다.

In [ ]:
# GRADED FUNCTION: compute_layer_style_cost

def compute_layer_style_cost(a_S, a_G):
    """
    Arguments:
    a_S -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing style of the image S 
    a_G -- tensor of dimension (1, n_H, n_W, n_C), hidden layer activations representing style of the image G
    
    Returns: 
    J_style_layer -- tensor representing a scalar value, style cost defined above by equation (2)
    """
    
    ### START CODE HERE ###
    # Retrieve dimensions from a_G (≈1 line)
    m, n_H, n_W, n_C = None
    
    # Reshape the images to have them of shape (n_C, n_H*n_W) (≈2 lines)
    a_S = None
    a_G = None

    # Computing gram_matrices for both images S and G (≈2 lines)
    GS = None
    GG = None

    # Computing the loss (≈1 line)
    J_style_layer = None
    
    ### END CODE HERE ###
    
    return J_style_layer

In [ ]:
tf.reset_default_graph()

with tf.Session() as test:
    tf.set_random_seed(1)
    a_S = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    a_G = tf.random_normal([1, 4, 4, 3], mean=1, stddev=4)
    J_style_layer = compute_layer_style_cost(a_S, a_G)
    
    print("J_style_layer = " + str(J_style_layer.eval()))

**모범 답안**:

<table>
    <tr>
        <td>
            <b>J_style_layer</b>
        </td>
        <td>
           9.19028
        </td>
    </tr>

</table>

#### 3.2.3 Style Weights

- 지금까지 하나의 은닉층에서만 style을 구했습니다.
- 여러 레이어의 style cost를 병합하면 더 나은 결과를 얻을 수 있습니다.
- 각 레이어에는 해당 레이어가 style cost에 기여하는 정도를 의미하는 가중치 ($\lambda^{[l]}$)가 부여됩니다.
- 이 과제를 완료한 후 다시 돌아와서 generated 이미지 $G$가 어떻게 변경되는지 다른 가중치로 실험 해보세요.
- 기본적으로 각 레이어에 동일한 가중치를 부여합니다. 가중치의 합은 1이됩니다. ($\sum_{l}^L \lambda^{[l]} = 1$)

In [ ]:
STYLE_LAYERS = [
    ('conv1_1', 0.2),
    ('conv2_1', 0.2),
    ('conv3_1', 0.2),
    ('conv4_1', 0.2),
    ('conv5_1', 0.2)]

다음과 같이 여러 레이어의 style cost를 결합할 수 있습니다.

$$ J_{style}(S, G) = \sum_{l} \lambda^{[l]} J^{[l]}_{style}(S, G) $$

$ \lambda^{[l]} $의 값은`STYLE_LAYERS`에 제공됩니다.

### 연습 문제 : style cost 계산

- 우리는 `compute_style_cost(...)` 함수를 구현했습니다.
- `compute_layer_style_cost(...)`를 여러 번 호출하고 `STYLE_LAYERS`의 값을 사용하여 결과에 가중치를 부여합니다.
- 위 과정이 뭘 의미하는지 이해하기 위해서는 아래 글을 쭉 읽어보세요.

<br>

**`compute_style_cost` 설명**
각 레이어에 대해 :
- 현재 레이어의 활성화 (출력 텐서)를 선택합니다.
- 현재 레이어에서 style 이미지 "S"의 스타일을 가져옵니다.
- 현재 레이어에서 generated 이미지 "G"의 스타일을 가져옵니다.
- 현재 레이어에 대한 style cost를 계산합니다
- 전체 style cost (J_style)에 가중치가 적용된 단일 레이어에서의 style cost를 더합니다.

루프를 완료하면 :
- 전체 style cost를 반환합니다.

In [ ]:
def compute_style_cost(model, STYLE_LAYERS):
    """
    Computes the overall style cost from several chosen layers
    
    Arguments:
    model -- our tensorflow model
    STYLE_LAYERS -- A python list containing:
                        - the names of the layers we would like to extract style from
                        - a coefficient for each of them
    
    Returns: 
    J_style -- tensor representing a scalar value, style cost defined above by equation (2)
    """
    
    # initialize the overall style cost
    J_style = 0

    for layer_name, coeff in STYLE_LAYERS:

        # Select the output tensor of the currently selected layer
        out = model[layer_name]

        # Set a_S to be the hidden layer activation from the layer we have selected, by running the session on out
        a_S = sess.run(out)

        # Set a_G to be the hidden layer activation from same layer. Here, a_G references model[layer_name] 
        # and isn't evaluated yet. Later in the code, we'll assign the image G as the model input, so that
        # when we run the session, this will be the activations drawn from the appropriate layer, with G as input.
        a_G = out
        
        # Compute style_cost for the current layer
        J_style_layer = compute_layer_style_cost(a_S, a_G)

        # Add coeff * J_style_layer of this layer to overall style cost
        J_style += coeff * J_style_layer

    return J_style

** 참고 ** : 위의 for 루프의 내부 루프에서 'a_G'는 텐서이며 아직 평가되지 않았습니다. 아래 `model_nn()` 에서 TensorFlow 그래프를 실행할 때 매 반복마다 평가되고 업데이트됩니다.


## 기억해야 할 사항
- 히든 레이어 활성화의 그램 행렬을 이용하여 이미지의 스타일을 표현할 수 있습니다.
- 이 표현을 여러 다른 레이어에서 결합하여 더 나은 결과를 얻습니다.
- 이것은 일반적으로 단일 히든 레이어를 사용하는 것으로 충분할 경우 콘텐츠 표현과는 대조적입니다.
- 스타일 비용을 최소화하면 $ G $ 이미지가 $ S $ 이미지의 스타일을 따르게됩니다.

### 3.3 - Defining the total cost to optimize

마지막으로 style, contents 비용을 모두 최소화하는 비용 함수를 만들어 보겠습니다. 공식은 다음과 같습니다.

$$ J (G) = \ alpha J_ {content} (C, G) + \ beta J_ {style} (S, G) $$

**연습 문제** : content cost와 style cost를 모두 포함하는 총 비용 함수를 구현하세요.

In [ ]:
# GRADED FUNCTION: total_cost

def total_cost(J_content, J_style, alpha = 10, beta = 40):
    """
    Computes the total cost function
    
    Arguments:
    J_content -- content cost coded above
    J_style -- style cost coded above
    alpha -- hyperparameter weighting the importance of the content cost
    beta -- hyperparameter weighting the importance of the style cost
    
    Returns:
    J -- total cost as defined by the formula above.
    """
    
    ### START CODE HERE ### (≈1 line)
    J = (alpha * J_content) + (beta * J_style)
    ### END CODE HERE ###
    
    return J

In [ ]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(3)
    J_content = np.random.randn()    
    J_style = np.random.randn()
    J = total_cost(J_content, J_style)
    print("J = " + str(J))

**모범 답안**:

<table>
    <tr>
        <td>
            <b>J</b>
        </td>
        <td>
           35.34667875478276
        </td>
    </tr>

</table>

## 기억해야 할 사항
- total cost는 content cost $J_{content}(C, G)$와 style cost $J_{style}(S, G)$ 의 일차 결합(선형 결합)입니다.
-$\alpha$ 및 $\beta$는 content와 style 간의 상대적 가중치를 제어하는 하이퍼 파라미터입니다.

## 4 - Solving the optimization problem


마지막으로 지금까지 구현한 모든 함수를 한데 모아 Neural Style Transfer을 완성해봅시다.

프로그램이 수행해야하는 작업은 다음과 같습니다.

1. Interactive Session 생성
2. 콘텐츠 이미지 불러오기
3. 스타일 이미지 불러오기
4. 생성 할 이미지를 임의로 초기화
5. VGG19 모델로드
6. TensorFlow 그래프를 만듭니다.
     - VGG19 모델을 통해 콘텐츠 이미지를 실행하고 콘텐츠 비용을 계산
     - VGG19 모델을 통해 스타일 이미지를 실행하고 스타일 비용을 계산
     - 총 비용 계산
     - 최적화 및 학습률 정의
7. TensorFlow 그래프를 초기화하고 여러 번 반복하여 실행하여 모든 단계에서 생성 된 이미지를 업데이트합니다.

개별 단계를 자세히 살펴 보겠습니다.

#### Interactive Sessions

이전에 총 비용 함수 $J(G)$를 구현했습니다. 이제 $G$와 관련하여 이를 최적화하기 위해 TensorFlow를 사용합니다.

- 이를 위해 프로그램에서 그래프를 재설정하고 "[Interactive Session](https://www.tensorflow.org/api_docs/python/tf/InteractiveSession)"을 사용해야합니다.
- 일반 세션과 달리 "Interactive Session"은 그래프를 작성하기위한 기본 세션으로 자체 설치됩니다.
- 이렇게하면 세션 객체 (`sess.run()` 호출)를 계속 참조 할 필요없이 변수를 실행할 수 있으므로 코드가 단순화됩니다.

#### Start the interactive session.

In [ ]:
# Reset the graph
tf.reset_default_graph()

# Start interactive session
sess = tf.InteractiveSession()

#### Content image

"content" 이미지인 루브르 박물관 사진을 불러와서 reshape, 정규화 해보겠습니다

In [ ]:
content_image = scipy.misc.imread("images/louvre_small.jpg")
content_image = reshape_and_normalize_image(content_image)

#### Style image

다음으로는 "style" 이미지인 클로드 모네의 작품 불러와서 위와 마찬가지로 reshape하고 정규화 해보겠습니다.

In [ ]:
style_image = scipy.misc.imread("images/monet.jpg")
style_image = reshape_and_normalize_image(style_image)

#### Generated image correlated with content image

이제 "generated" 이미지를 content_image에서 생성 된 노이즈 이미지로 초기화합니다.

- 생성 된 이미지는 콘텐츠 이미지와 약간의 상관 관계가 있습니다.
- 생성 된 이미지의 픽셀은 대부분 노이즈이지만 콘텐츠 이미지와 약간의 상관 관계가 있도록 초기화하면 "생성 된"이미지의 콘텐츠가 "콘텐츠"이미지의 콘텐츠와 더 빠르게 일치하는 데 도움이됩니다.
- `generate_noise_image(...)` 의 세부 사항을 보려면 `nst_utils.py` 에서 자유롭게 살펴보십시오. Jupyter 노트북의 왼쪽 상단 모서리에있는 "파일-> 열기 ..."를 클릭하면 됩니다.

In [ ]:
generated_image = generate_noise_image(content_image)
imshow(generated_image[0]);

#### Load pre-trained VGG19 model

part (2)에서 설명했던대로, VGG 19 모델을 불러오겠습니다.

In [ ]:
model = load_vgg_model("pretrained-model/imagenet-vgg-verydeep-19.mat")

#### Contents cost

프로그램이 content cost를 계산하도록하기 위해 `a_C`및 `a_G`를 적절한 히든 레이어의 활성화 변수로 할당합니다. 콘텐츠 비용을 계산하기 위해 'conv4_2'레이어를 사용합니다. 아래 코드는 다음을 수행합니다.

1. VGG 모델에 입력 할 콘텐츠 이미지를 할당합니다.
2. a_C를 텐서로 설정하여 "conv4_2"레이어에 대한 은닉 레이어 활성화를 제공합니다.
3. a_G를 동일한 레이어에 대해 은닉 레이어 활성화를 제공하는 텐서로 설정합니다.
4. a_C 및 a_G를 사용하여 콘텐츠 비용을 계산합니다.

**참고** : 이 시점에서 a_G는 텐서이며 평가되지 않았습니다. 아래의 `model_nn()` 에서 Tensorflow 그래프를 실행할 때 매 반복마다 평가되고 업데이트됩니다.

In [ ]:
# Assign the content image to be the input of the VGG model.  
sess.run(model['input'].assign(content_image))

# Select the output tensor of layer conv4_2
out = model['conv4_2']

# Set a_C to be the hidden layer activation from the layer we have selected
a_C = sess.run(out)

# Set a_G to be the hidden layer activation from same layer. Here, a_G references model['conv4_2'] 
# and isn't evaluated yet. Later in the code, we'll assign the image G as the model input, so that
# when we run the session, this will be the activations drawn from the appropriate layer, with G as input.
a_G = out

# Compute the content cost
J_content = compute_content_cost(a_C, a_G)

#### Style cost

In [ ]:
# Assign the input of the model to be the "style" image 
sess.run(model['input'].assign(style_image))

# Compute the style cost
J_style = compute_style_cost(model, STYLE_LAYERS)

### 연습 문제 : total cost

- 이제 J_content 및 J_style이 있으므로 `total_cost ()` 를 호출하여 총 비용 J를 계산합니다.
- `alpha = 10` 및`beta = 40`을 사용하십시오.

In [ ]:
### START CODE HERE ### (1 line)
J = None
### END CODE HERE ###

### Optimizer

* Adam 최적화 프로그램을 사용하여 총 비용 'J'를 최소화하십시오.
* 2.0의 학습률을 사용하십시오.
* [Adam Optimizer 문서](https://www.tensorflow.org/api_docs/python/tf/train/AdamOptimizer)

In [ ]:
# define optimizer (1 line)
optimizer = tf.train.AdamOptimizer(2.0)

# define train_step (1 line)
train_step = optimizer.minimize(J)

### Exercise : implement the model

- `model_nn()` 함수를 구현하십시오.
- 이 함수는 tensorflow 그래프의 변수를 **초기화**합니다.
- 입력 이미지(초기 생성 이미지)를 VGG19 모델의 입력으로 **할당** 합니다
- 그리고 `train_step` 텐서(이 함수 위의 코드에서 생성됨)를 많은 단계에 대해서 **실행**합니다.

#### Hints
* 전역 변수를 초기화하려면 다음을 사용하십시오.
```python
sess.run(tf.global_variables_initializer())
```
* 변수를 평가하려면 `sess.run()` 을 실행하십시오.
* [assign](https://www.tensorflow.org/versions/r1.14/api_docs/python/tf/assign)은 다음과 같이 사용할 수 있습니다.
```python
model["input"].assign(image)
```

In [ ]:
def model_nn(sess, input_image, num_iterations = 200):
    
    # Initialize global variables (you need to run the session on the initializer)
    ### START CODE HERE ### (1 line)
    None
    ### END CODE HERE ###
    
    # Run the noisy input image (initial generated image) through the model. Use assign().
    ### START CODE HERE ### (1 line)
    None
    ### END CODE HERE ###
    
    for i in range(num_iterations):
    
        # Run the session on the train_step to minimize the total cost
        ### START CODE HERE ### (1 line)
        None
        ### END CODE HERE ###
        
        # Compute the generated image by running the session on the current model['input']
        ### START CODE HERE ### (1 line)
        generated_image = None
        ### END CODE HERE ###

        # Print every 20 iteration.
        if i%20 == 0:
            Jt, Jc, Js = sess.run([J, J_content, J_style])
            print("Iteration " + str(i) + " :")
            print("total cost = " + str(Jt))
            print("content cost = " + str(Jc))
            print("style cost = " + str(Js))
            
            # save current generated image in the "/output" directory
            save_image("output/" + str(i) + ".png", generated_image)
    
    # save last generated image
    save_image('output/generated_image.jpg', generated_image)
    
    return generated_image

다음 셀을 실행하여 예술 작품 이미지를 생성합니다. 20회 반복 할 때마다 CPU에서 약 3분이 소요되고, 약 140 회 반복 후에 매력적인 결과를 출력하기 시작합니다. Neural Style Transfer는 일반적으로 GPU를 사용하여 훈련됩니다.

In [ ]:
model_nn(sess, generated_image)

**모범 답안**:

<table>
    <tr>
        <td>
            <b>Iteration 0 : </b>
        </td>
        <td>
           total cost = 5.05035e+09 <br>
           content cost = 7877.67 <br>
           style cost = 1.26257e+08
        </td>
    </tr>

</table>

끝났습니다! 이를 실행 한 후 노트북 상단 표시 줄에서 "파일"을 클릭 한 다음 "열기"를 클릭합니다. 저장된 모든 이미지를 보려면 "/output" 디렉토리로 이동하십시오. 생성 된 이미지를 보려면 "generated_image" 를 여십시오! :)

오른쪽에 아래에 표시된 이미지가 표시되어야합니다.

<img src = "images / louvre_generated.png"style = "width : 800px; height : 300px;">

초기 결과를보기 위해 너무 오래 기다리지 않았으므로 이에 따라 하이퍼 파라미터를 설정했습니다. 최상의 결과를 얻으려면 최적화 알고리즘을 더 오래 (그리고 아마도 더 작은 학습률로) 실행하는 것이 더 효과적 일 수 있습니다. 이 과제를 완료하고 제출 한 후 다시 돌아와서이 노트북을 더 많이 사용하고 더보기 좋은 이미지를 생성 할 수 있는지 확인하시기 바랍니다.

다음은 몇 가지 다른 예입니다.

- 반 고흐 (별이 빛나는 밤) 스타일의 고대 도시 페르 세 폴리스 (이란)의 아름다운 유적

<img src="arts/perspolis_vangogh.png" style="width:750px;height:300px;">

- 이스파한의 도자기 카시 양식으로 파 사르가 대에있는 고레스 대왕의 무덤.

<img src="arts/pasargad_kashi.png" style="width:750px;height:300px;">

- 추상적 인 파란색 유체 페인팅 스타일의 난류 유체에 대한 과학적 연구.

<img src="arts/circle_abstract.png" style="width:750px;height:300px;">

## 5 - Test with your own Image(Optional/Ungraded)

마지막으로 자신의 이미지에서 알고리즘을 다시 실행할 수도 있습니다!

파트 4로 돌아가서 자신의 사진으로 콘텐츠 이미지와 스타일 이미지를 변경합니다. 자세한 내용은 다음과 같습니다.

1. 노트북 상단 탭에서 "파일 -> 열기"를 클릭합니다.
2. "/images"로 이동하여 이미지를 업로드하고 (규격:(WIDTH=300, HEIGHT=225)), 예를 들어 "my_content.png" 및 "my_style.png"로 이름을 바꿉니다.
3. (3.4) 부분의 코드를 다음에서 변경합니다.
```python
content_image = scipy.misc.imread ("images/louvre.jpg")
style_image = scipy.misc.imread ("images/claude-monet.jpg")
```
에:
```python
content_image = scipy.misc.imread ("images/my_content.jpg")
style_image = scipy.misc.imread ("images/my_style.jpg")
```
4. 셀을 다시 실행합니다 (노트북의 상단 탭에서 커널을 다시 시작해야 할 수 있음).

생성 된 이미지를 해시 태그 #deeplearniNgAI 또는 직접 태그를 통해 소셜 미디어에서 공유 할 수 있습니다!

하이퍼 파라미터를 조정할 수도 있습니다.
- 스타일을 나타내는 레이어는 무엇입니까? **STYLE_LAYERS**
- 알고리즘을 몇 번 반복 하시겠습니까? **num_iterations**
- 콘텐츠와 스타일 사이의 상대적 가중치는 무엇입니까? **alpha/beta**

## 6 - Conclusion

이 과제를 완료했습니다. 이제 Neural Style Transfer를 사용하여 예술 작품 이미지를 생성 할 수 있습니다. 최적화 알고리즘이 신경망의 파라미터가 아닌 픽셀 값을 업데이트하는 모델을 구축하는 것도 이번이 처음입니다. 딥 러닝에는 다양한 유형의 모델이 있으며 이것은 그중 하나 일뿐입니다!

## 기억해야 할 사항
- Neural Style Transfer는 콘텐츠 이미지 C와 스타일 이미지 S가 주어지면 예술적 이미지를 생성 할 수있는 알고리즘입니다.
- 사전 훈련 된 ConvNet을 기반으로 한 representation(은닉층의 활성화)을 사용합니다.
- content cost function는 하나의 히든 레이어 활성화를 사용하여 계산됩니다.
- 한 레이어의 style cost function은 해당 레이어의 활성화에 대한 Gram matrix을 사용하여 계산됩니다. total cost function은 여러 숨겨진 레이어를 사용하여 얻습니다.
- total cost function을 최적화하면 새로운 이미지가 합성됩니다.

### References:

The Neural Style Transfer algorithm was due to Gatys et al. (2015). Harish Narayanan and Github user "log0" also have highly readable write-ups from which we drew inspiration. The pre-trained network used in this implementation is a VGG network, which is due to Simonyan and Zisserman (2015). Pre-trained weights were from the work of the MathConvNet team. 

- Leon A. Gatys, Alexander S. Ecker, Matthias Bethge, (2015). [A Neural Algorithm of Artistic Style](https://arxiv.org/abs/1508.06576) 
- Harish Narayanan, [Convolutional neural networks for artistic style transfer.](https://harishnarayanan.org/writing/artistic-style-transfer/)
- Log0, [TensorFlow Implementation of "A Neural Algorithm of Artistic Style".](http://www.chioka.in/tensorflow-implementation-neural-algorithm-of-artistic-style)
- Karen Simonyan and Andrew Zisserman (2015). [Very deep convolutional networks for large-scale image recognition](https://arxiv.org/pdf/1409.1556.pdf)
- [MatConvNet.](http://www.vlfeat.org/matconvnet/pretrained/)